In [ ]:
import pandas as pd
import os
import pytz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
import shutil
import csv

from datetime import datetime, timedelta
from dateutil import rrule

In [ ]:
data_folder = '../data/water.csv'
water_file = '../data/water.csv'

# Specify the current delimiter in water
current_delimiter = ';'  

with open(water_file, 'r', newline='') as file:
    # Create a CSV reader with the current delimiter
    reader = csv.reader(file, delimiter=current_delimiter)

    # Read the data from the input file
    data = [row for row in reader]

# Write the csv file with comma as delimiter
with open(data_folder, 'w', newline='') as output_file:
    
    # Create a CSV writer with a comma as the delimiter
    writer = csv.writer(output_file, delimiter=',')

    # Write the data to the output file
    writer.writerows(data)


In [ ]:
# Change the date frame     
df = pd.read_csv(water_file)
df.rename(columns={'DateTime':'datumBeginMeting','EDE_09902MTW_K100.MTW':'hstWaarde'},inplace=True)

df['hstWaarde'] = df['hstWaarde'].replace(['(null)', ''], np.nan).str.replace(',', '.', regex=True).astype(float)

df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'], dayfirst=True)

output_file_path = os.path.join('../data/', 'water.csv')

df.to_csv(output_file_path, index=False)

df


In [ ]:
data_dir = '../data/'
csv_output_dir = '../data/important/'

all_files = os.listdir(data_dir)

parquet_files = [file for file in all_files if file.endswith('.parquet')]

# Loop through each Parquet file and convert it to CSV
for parquet_file in parquet_files:
    # Construct the full path to the Parquet file
    parquet_path = os.path.join(data_dir, parquet_file)
    
    # Load the Parquet file into a DataFrame
    df = pd.read_parquet(parquet_path)
    
    # Specify the CSV file name by replacing the ".parquet" extension with ".csv"
    csv_file = parquet_file.replace('.parquet', '.csv')
    
    # Construct the full path to the CSV file in the output directory
    csv_path = os.path.join(csv_output_dir, csv_file)
    
    if not os.path.exists('../data/important/'):
        os.makedirs('../data/important/')
    
    shutil.copy(f'{data_dir}water.csv', csv_output_dir) 
    # Save the DataFrame to the CSV file in the output directory
    df.to_csv(csv_path, index=False)


In [ ]:
'''
Function to cycle through the folder of Ammonium, Nitrate, Oxygen_A, Oxygen_B and Phosphate CSVs and convert them to Series.

Return: Pandas Series for each above mentioned
'''

# change the relative path according
folder_path = "../data/important/"
convert_dir = '../data/converted/'

files = os.listdir(folder_path)

for file_name in files:
    df = pd.read_csv(f"../data/important/{file_name}")

    df.index = pd.to_datetime(df['datumBeginMeting']).dt.strftime('%Y-%m-%d %H:%M')
    df.index.name = None
    
    if 'datumEindeMeting' in df.columns:
        df.drop(columns=['datumEindeMeting', 'datumBeginMeting'], inplace=True)
        df_to_series = df["hstWaarde"]
        filename = convert_dir + df['historianTagnummer'].iloc[0].split('.')[0] + f'_{file_name}'
    else:
        df.drop(columns=['datumBeginMeting'], inplace=True)
    
        df_to_series = df["hstWaarde"]
        filename = convert_dir + f'_{file_name}'
    
    if not os.path.exists('../data/converted/'):
        os.makedirs('../data/converted/')
    
    df_to_series.to_csv(filename, header=True)
    df_to_series

In [ ]:
'''
Return: Pandas Series for each above mentioned
'''
folder_path = "../data/converted/"
files = os.listdir(folder_path)

for file_name in files:
    df = pd.read_csv(f"../data/converted/{file_name}", parse_dates=True, index_col=0)  

    df.index = pd.to_datetime(df.index)

    minutely_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='T')
    new_df = pd.DataFrame(index=minutely_index)

    merged_df = new_df.join(df, how='left').interpolate(method='time')

    merged_df = merged_df.groupby(merged_df.index).mean()
    
    if not os.path.exists('../data/resampled/'):
        os.makedirs('../data/resampled/')
        
    merged_df.to_csv(f"../data/resampled/{file_name}")

In [ ]:
folder_path = "../data/resampled/"
files = os.listdir(folder_path)

merged = pd.DataFrame()

for file_name in files:
    df = pd.read_csv(f"../data/resampled/{file_name}", index_col=0)
    df.index = pd.to_datetime(df.index)
    # df = df.resample('2min').mean()
    df = df.rename(columns={"hstWaarde": file_name})
    merged = pd.merge(df, merged, left_index=True, right_index=True, how='outer')

merged.to_csv("../data/merged.csv")